# Insurance Analytics: Medallion Architecture with Delta Liquid Clustering



## Overview



This notebook demonstrates a complete **Medallion Architecture** implementation in Oracle AI Data Platform (AIDP) Workbench for insurance analytics, incorporating **Delta Liquid Clustering** for optimal performance. The medallion architecture organizes data into Bronze, Silver, and Gold layers for progressive data refinement and business value creation.



### Medallion Architecture Overview



**Bronze Layer**: Raw data ingestion with minimal processing - preserves original data fidelity



**Silver Layer**: Cleaned, standardized, and enriched data - ready for analysis



**Gold Layer**: Curated, aggregated data for analytics, ML-ready datasets, and business metrics



### Delta Liquid Clustering



Liquid clustering automatically identifies and groups similar data together based on clustering columns you define. This optimization happens automatically during data ingestion and maintenance operations, providing:



- **Automatic optimization**: No manual tuning required

- **Improved query performance**: Faster queries on clustered columns

- **Reduced maintenance**: No need for manual repartitioning

- **Adaptive clustering**: Adjusts as data patterns change



### Use Case: Insurance Risk Assessment and Fraud Detection



We'll process insurance claim data through the medallion layers, optimizing each layer for specific analytical needs:



- **Bronze**: Raw claim ingestion with liquid clustering on `policy_id` and `claim_date`

- **Silver**: Data quality checks, standardization, and enrichment

- **Gold**: Aggregated analytics and ML-ready datasets for fraud detection



### AIDP Environment Setup



This notebook leverages the existing Spark session in your AIDP environment.

In [1]:
# Setup: Create insurance catalog and schemas for medallion architecture

# In AIDP, catalogs provide data isolation and governance

spark.sql("CREATE CATALOG IF NOT EXISTS insurance")

spark.sql("CREATE SCHEMA IF NOT EXISTS insurance.bronze")

spark.sql("CREATE SCHEMA IF NOT EXISTS insurance.silver")

spark.sql("CREATE SCHEMA IF NOT EXISTS insurance.gold")

print("Insurance catalog and bronze/silver/gold schemas created successfully!")

Insurance catalog and bronze/silver/gold schemas created successfully!


# Bronze Layer: Raw Data Ingestion



## Bronze Layer Purpose



The Bronze layer serves as the raw data ingestion point. Data is ingested with minimal transformation to preserve:



- **Data fidelity**: Original data as received from source systems

- **Auditability**: Complete historical record

- **Reprocessing capability**: Ability to reprocess data if business rules change



### Bronze Layer Characteristics



- **Append-only**: New data is appended, existing data is never modified

- **Raw format**: Minimal schema enforcement

- **Timestamped**: Includes ingestion timestamps

- **Optimized for ingestion**: Fast write performance

## Step 1: Create Bronze Layer Table with Liquid Clustering



### Table Design for Bronze Layer



Our bronze `insurance_claims_raw` table will store raw claim data with:


- **Raw fields**: All original data fields preserved
- **Ingestion metadata**: Timestamps and source information
- **Liquid clustering**: Optimized for `policy_id` and `ingestion_date` for efficient querying



### Clustering Strategy for Bronze


We'll cluster by `policy_id` and `ingestion_date` because:


- **policy_id**: Groups all claims for the same policy together
- **ingestion_date**: Enables time-based data management and reprocessing
- This combination optimizes for both policy analysis and temporal operations

In [11]:
# Create Bronze layer Delta table with liquid clustering

# CLUSTER BY defines the columns for automatic optimization

spark.sql("""

CREATE TABLE IF NOT EXISTS insurance.bronze.insurance_claims_raw (

    -- Raw claim data fields

    policy_id STRING,

    claim_date TIMESTAMP,

    claim_type STRING,

    claim_amount DECIMAL(15,2),

    incident_type STRING,

    location STRING,

    fraud_score INT,

    

    -- Bronze layer metadata

    ingestion_timestamp TIMESTAMP,

    ingestion_date DATE,

    source_system STRING,

    batch_id STRING

)

USING DELTA

CLUSTER BY (policy_id, ingestion_date)

""")

print("Bronze layer Delta table with liquid clustering created successfully!")

print("Clustering will automatically optimize data layout for queries on policy_id and ingestion_date.")

Bronze layer Delta table with liquid clustering created successfully!
Clustering will automatically optimize data layout for queries on policy_id and ingestion_date.


## Step 2: Generate and Ingest Raw Insurance Data



### Data Generation Strategy


We'll create realistic insurance claim data simulating multiple data sources:


- **Multiple batches**: Simulating daily data loads
- **Source systems**: Different claim processing systems
- **Realistic patterns**: Seasonal variations, system-specific data quality



### Bronze Layer Ingestion


Data is ingested in batches with:

- **Batch IDs**: For tracking and reprocessing
- **Source metadata**: System provenance
- **Ingestion timestamps**: Processing audit trail

In [12]:
# Generate sample insurance claim data for Bronze layer ingestion

# Using fully qualified imports to avoid conflicts

import random
import uuid
from datetime import datetime, timedelta
from decimal import Decimal


# Define insurance data constants

CLAIM_TYPES = ['Auto', 'Home', 'Health', 'Life', 'Property']
INCIDENT_TYPES = ['Accident', 'Theft', 'Natural Disaster', 'Illness', 'Fire', 'Flood', 'Collision', 'Medical Emergency']
LOCATIONS = ['New York, NY', 'Los Angeles, CA', 'Chicago, IL', 'Houston, TX', 'Miami, FL', 'Denver, CO', 'Seattle, WA']
SOURCE_SYSTEMS = ['Legacy_Claims', 'Mobile_App', 'Partner_API', 'Direct_Web']

# Generate raw claim records for multiple batches
raw_claim_data = []
base_date = datetime(2024, 1, 1)

# Simulate 5 days of data ingestion
for batch_day in range(5):
    batch_date = base_date + timedelta(days=batch_day)
    batch_id = f"BATCH_{batch_date.strftime('%Y%m%d')}"
    
    # Each batch has different volumes from different sources
    for source_system in SOURCE_SYSTEMS:
        # Vary batch sizes by source system
        if source_system == 'Legacy_Claims':
            batch_size = random.randint(800, 1200)
        elif source_system == 'Mobile_App':
            batch_size = random.randint(200, 400)
        elif source_system == 'Partner_API':
            batch_size = random.randint(300, 600)
        else:  # Direct_Web
            batch_size = random.randint(100, 300)
        
        for i in range(batch_size):
            policy_num = random.randint(1, 10000)
            policy_id = f"POL{policy_num:08d}"
            
            # Spread claims over time
            claim_days_offset = random.randint(0, 365)
            claim_hours_offset = random.randint(0, 23)
            claim_date = base_date + timedelta(days=claim_days_offset, hours=claim_hours_offset)
            
            # Select claim attributes
            claim_type = random.choice(CLAIM_TYPES)
            
            # Amount based on claim type (with some data quality variation by source)
            if claim_type == 'Auto':
                amount = round(random.uniform(1000, 50000), 2)
            elif claim_type == 'Home':
                amount = round(random.uniform(5000, 200000), 2)
            elif claim_type == 'Health':
                amount = round(random.uniform(500, 100000), 2)
            elif claim_type == 'Life':
                amount = round(random.uniform(10000, 500000), 2)
            else:  # Property
                amount = round(random.uniform(2000, 150000), 2)
            
            # Add some data quality issues for realism
            if source_system == 'Legacy_Claims' and random.random() < 0.1:
                amount = amount * -1  # Negative amounts in legacy data
            
            incident_type = random.choice(INCIDENT_TYPES)
            location = random.choice(LOCATIONS)
            fraud_score = random.randint(0, 100)
            
            # Ingestion metadata
            ingestion_timestamp = batch_date + timedelta(hours=random.randint(0, 23), minutes=random.randint(0, 59))
            
            raw_claim_data.append({
                "policy_id": policy_id,
                "claim_date": claim_date,
                "claim_type": claim_type,
                "claim_amount": Decimal(amount),
                "incident_type": incident_type,
                "location": location,
                "fraud_score": fraud_score,
                "ingestion_timestamp": ingestion_timestamp,
                "ingestion_date": batch_date.date(),
                "source_system": source_system,
                "batch_id": batch_id
            })

print(f"Generated {len(raw_claim_data)} raw insurance claim records across {len(SOURCE_SYSTEMS) * 5} batches")
print(f"Data sources: {SOURCE_SYSTEMS}")
if raw_claim_data:
    print("Sample record:", raw_claim_data[0])

Generated 9666 raw insurance claim records across 20 batches
Data sources: ['Legacy_Claims', 'Mobile_App', 'Partner_API', 'Direct_Web']
Sample record: {'policy_id': 'POL00003296', 'claim_date': datetime.datetime(2024, 3, 6, 10, 0), 'claim_type': 'Home', 'claim_amount': Decimal('158287.3099999999976716935634613037109375'), 'incident_type': 'Fire', 'location': 'Los Angeles, CA', 'fraud_score': 52, 'ingestion_timestamp': datetime.datetime(2024, 1, 1, 17, 56), 'ingestion_date': datetime.date(2024, 1, 1), 'source_system': 'Legacy_Claims', 'batch_id': 'BATCH_20240101'}


In [17]:
# Insert raw data into Bronze layer
from pyspark.sql.functions import col
from pyspark.sql.types import DecimalType, IntegerType

# Create DataFrame from raw data

df_raw_claims = spark.createDataFrame(raw_claim_data)


print("Bronze Layer DataFrame Schema:")
df_raw_claims.printSchema()

print("\nSample Bronze Layer Data:")
df_raw_claims.show(5)

df_raw_claims = df_raw_claims.withColumn("claim_amount", col("claim_amount").cast(DecimalType(15, 2)))
df_raw_claims = df_raw_claims.withColumn("fraud_score", col("fraud_score").cast(IntegerType()))

# Insert data into Bronze layer with liquid clustering
# The CLUSTER BY (policy_id, ingestion_date) will automatically optimize the data layout

df_raw_claims.write.mode("append").saveAsTable("insurance.bronze.insurance_claims_raw")

print(f"\nSuccessfully inserted {df_raw_claims.count()} raw records into insurance.bronze.insurance_claims_raw")
print("Liquid clustering automatically optimized the data layout during ingestion!")

# Verify Bronze layer data
bronze_count = spark.sql("SELECT COUNT(*) as total_records FROM insurance.bronze.insurance_claims_raw").collect()[0][0]
print(f"\nBronze layer now contains {bronze_count} total records")

Bronze Layer DataFrame Schema:
root
 |-- batch_id: string (nullable = true)
 |-- claim_amount: decimal(38,18) (nullable = true)
 |-- claim_date: timestamp (nullable = true)
 |-- claim_type: string (nullable = true)
 |-- fraud_score: long (nullable = true)
 |-- incident_type: string (nullable = true)
 |-- ingestion_date: date (nullable = true)
 |-- ingestion_timestamp: timestamp (nullable = true)
 |-- location: string (nullable = true)
 |-- policy_id: string (nullable = true)
 |-- source_system: string (nullable = true)


Sample Bronze Layer Data:
+--------------+--------------------+-------------------+----------+-----------+-----------------+--------------+-------------------+---------------+-----------+-------------+
|      batch_id|        claim_amount|         claim_date|claim_type|fraud_score|    incident_type|ingestion_date|ingestion_timestamp|       location|  policy_id|source_system|
+--------------+--------------------+-------------------+----------+-----------+---------------


Successfully inserted 9666 raw records into insurance.bronze.insurance_claims_raw
Liquid clustering automatically optimized the data layout during ingestion!



Bronze layer now contains 9666 total records


# Silver Layer: Data Cleaning and Enrichment



## Silver Layer Purpose



The Silver layer transforms raw Bronze data into clean, standardized, and enriched datasets:



- **Data quality**: Validation, cleansing, and standardization
- **Business rules**: Application of business logic and transformations
- **Enrichment**: Addition of derived fields and external data
- **Deduplication**: Removal of duplicate records



### Silver Layer Characteristics


- **Refined data**: Clean and standardized
- **Business-aligned**: Optimized for analytical use cases
- **Versioned**: Tracks transformation logic versions
- **Queryable**: Supports both operational and analytical queries

## Step 3: Create Silver Layer Table with Liquid Clustering



### Silver Layer Design



Our silver `insurance_claims_clean` table will include:


- **Cleaned fields**: Validated and standardized data
- **Derived fields**: Calculated metrics and categorizations
- **Quality flags**: Data quality indicators
- **Business metadata**: Processing information



### Clustering Strategy for Silver


We'll cluster by `policy_id` and `claim_date` because:


- **policy_id**: Enables fast policy-level analytics
- **claim_date**: Optimizes time-series analysis and fraud detection
- This combination supports both operational and analytical workloads

In [18]:
# Create Silver layer Delta table with liquid clustering

spark.sql("""

CREATE TABLE IF NOT EXISTS insurance.silver.insurance_claims_clean (

    -- Core claim data (cleaned and standardized)

    policy_id STRING,

    claim_date TIMESTAMP,

    claim_type STRING,

    claim_amount DECIMAL(15,2),

    incident_type STRING,

    location STRING,

    fraud_score INT,

    

    -- Derived and enriched fields

    claim_year INT,

    claim_month INT,

    claim_day_of_week INT,

    claim_hour INT,

    risk_category STRING,

    amount_category STRING,

    is_high_value_claim BOOLEAN,

    

    -- Data quality and processing metadata

    data_quality_score DOUBLE,

    processing_timestamp TIMESTAMP,

    silver_version STRING,

    source_system STRING,

    batch_id STRING

)

USING DELTA

CLUSTER BY (policy_id, claim_date)

""")

print("Silver layer Delta table with liquid clustering created successfully!")
print("Clustering will optimize for policy-based and temporal analytical queries.")

Silver layer Delta table with liquid clustering created successfully!
Clustering will optimize for policy-based and temporal analytical queries.


## Step 4: Transform Bronze to Silver Layer



### Data Transformation Logic


Transformations applied from Bronze to Silver:


1. **Data Quality Checks**: Validate required fields and data ranges
2. **Standardization**: Normalize formats and values
3. **Deduplication**: Remove duplicate records based on business keys
4. **Enrichment**: Add derived fields and categorizations
5. **Business Rules**: Apply insurance-specific logic



### Quality Assurance


- **Completeness**: Ensure all required fields are present
- **Accuracy**: Validate data ranges and formats
- **Consistency**: Standardize categorical values
- **Timeliness**: Check for reasonable date ranges

In [20]:
# Transform Bronze data to Silver layer

from pyspark.sql.functions import (
    col, when, year, month, dayofweek, hour, 
    abs, round, concat, lit, current_timestamp
)
from pyspark.sql.window import Window
from pyspark.sql import functions as F

# Read Bronze layer data
df_bronze = spark.read.table("insurance.bronze.insurance_claims_raw")

print(f"Processing {df_bronze.count()} records from Bronze layer")

# Apply data quality and transformation logic
df_silver = df_bronze \
    .withColumn("claim_amount", abs(col("claim_amount"))) \
    .withColumn("claim_year", year(col("claim_date"))) \
    .withColumn("claim_month", month(col("claim_date"))) \
    .withColumn("claim_day_of_week", dayofweek(col("claim_date"))) \
    .withColumn("claim_hour", hour(col("claim_date"))) \
    .withColumn("risk_category", 
        when(col("fraud_score") >= 80, "Very High Risk")
        .when(col("fraud_score") >= 60, "High Risk")
        .when(col("fraud_score") >= 40, "Medium Risk")
        .when(col("fraud_score") >= 20, "Low Risk")
        .otherwise("Very Low Risk")
    ) \
    .withColumn("amount_category",
        when(col("claim_amount") >= 100000, "High Value")
        .when(col("claim_amount") >= 25000, "Medium Value")
        .otherwise("Low Value")
    ) \
    .withColumn("is_high_value_claim", col("claim_amount") >= 50000) \
    .withColumn("data_quality_score", 
        when(col("policy_id").isNotNull() & 
             col("claim_date").isNotNull() & 
             (col("claim_amount") > 0), 1.0)
        .otherwise(0.5)
    ) \
    .withColumn("processing_timestamp", current_timestamp()) \
    .withColumn("silver_version", lit("v1.0"))

# Remove duplicates based on policy_id, claim_date, claim_type, claim_amount
window_spec = Window.partitionBy("policy_id", "claim_date", "claim_type", "claim_amount") \
                 .orderBy(col("ingestion_timestamp").desc())

df_silver_deduped = df_silver \
    .withColumn("row_num", F.row_number().over(window_spec)) \
    .filter(col("row_num") == 1) \
    .drop("row_num")

# Select final Silver layer columns
df_silver_final = df_silver_deduped.select(
    "policy_id", "claim_date", "claim_type", "claim_amount",
    "incident_type", "location", "fraud_score",
    "claim_year", "claim_month", "claim_day_of_week", "claim_hour",
    "risk_category", "amount_category", "is_high_value_claim",
    "data_quality_score", "processing_timestamp", "silver_version",
    "source_system", "batch_id"
)

print(f"After transformations: {df_silver_final.count()} clean records")
print("\nSilver Layer Schema:")
df_silver_final.printSchema()

print("\nSample Silver Layer Data:")
df_silver_final.show(5)

Processing 9666 records from Bronze layer


After transformations: 9666 clean records

Silver Layer Schema:
root
 |-- policy_id: string (nullable = true)
 |-- claim_date: timestamp (nullable = true)
 |-- claim_type: string (nullable = true)
 |-- claim_amount: decimal(15,2) (nullable = true)
 |-- incident_type: string (nullable = true)
 |-- location: string (nullable = true)
 |-- fraud_score: integer (nullable = true)
 |-- claim_year: integer (nullable = true)
 |-- claim_month: integer (nullable = true)
 |-- claim_day_of_week: integer (nullable = true)
 |-- claim_hour: integer (nullable = true)
 |-- risk_category: string (nullable = false)
 |-- amount_category: string (nullable = false)
 |-- is_high_value_claim: boolean (nullable = true)
 |-- data_quality_score: double (nullable = false)
 |-- processing_timestamp: timestamp (nullable = false)
 |-- silver_version: string (nullable = false)
 |-- source_system: string (nullable = true)
 |-- batch_id: string (nullable = true)


Sample Silver Layer Data:


+-----------+-------------------+----------+------------+-------------+---------------+-----------+----------+-----------+-----------------+----------+-------------+---------------+-------------------+------------------+--------------------+--------------+-------------+--------------+
|  policy_id|         claim_date|claim_type|claim_amount|incident_type|       location|fraud_score|claim_year|claim_month|claim_day_of_week|claim_hour|risk_category|amount_category|is_high_value_claim|data_quality_score|processing_timestamp|silver_version|source_system|      batch_id|
+-----------+-------------------+----------+------------+-------------+---------------+-----------+----------+-----------+-----------------+----------+-------------+---------------+-------------------+------------------+--------------------+--------------+-------------+--------------+
|POL00000001|2024-10-31 17:00:00|      Auto|    30735.20|         Fire|Los Angeles, CA|          8|      2024|         10|                5|  

In [21]:
# Insert transformed data into Silver layer

df_silver_final.write.mode("overwrite").saveAsTable("insurance.silver.insurance_claims_clean")

print(f"Successfully inserted {df_silver_final.count()} cleaned records into insurance.silver.insurance_claims_clean")

# Verify Silver layer data
silver_count = spark.sql("SELECT COUNT(*) as total_records FROM insurance.silver.insurance_claims_clean").collect()[0][0]
quality_stats = spark.sql("""
    SELECT 
        ROUND(AVG(data_quality_score), 3) as avg_quality_score,
        COUNT(*) as total_claims,
        COUNT(DISTINCT policy_id) as unique_policies
    FROM insurance.silver.insurance_claims_clean
""").collect()[0]

print(f"\nSilver layer verification:")
print(f"- Total records: {silver_count}")
print(f"- Average data quality score: {quality_stats[0]}")
print(f"- Unique policies: {quality_stats[2]}")

Successfully inserted 9666 cleaned records into insurance.silver.insurance_claims_clean



Silver layer verification:
- Total records: 9666
- Average data quality score: 1.0
- Unique policies: 6228


# Gold Layer: Analytics and ML-Ready Data



## Gold Layer Purpose



The Gold layer provides curated, aggregated datasets optimized for:


- **Business intelligence**: Pre-aggregated metrics and KPIs
- **Machine learning**: Feature engineering and model-ready datasets
- **Real-time analytics**: Optimized for dashboard and reporting queries
- **Data products**: Clean, governed datasets for downstream consumption



### Gold Layer Characteristics


- **Aggregated data**: Summarized metrics and KPIs
- **ML-ready**: Feature-engineered datasets for predictive modeling
- **Optimized**: Indexed and partitioned for query performance
- **Governed**: Well-documented and versioned

## Step 5: Create Gold Layer Analytics Tables



### Gold Layer Design



We'll create two Gold layer tables:


1. **Claim Analytics**: Aggregated business metrics
2. **Fraud ML Dataset**: Feature-engineered data for fraud detection models


Both tables will use liquid clustering optimized for their specific query patterns.

In [24]:
# Create Gold layer analytics table

spark.sql("""

CREATE TABLE IF NOT EXISTS insurance.gold.claim_analytics (

    -- Aggregation dimensions

    claim_year INT,

    claim_month INT,

    claim_type STRING,

    incident_type STRING,

    location STRING,

    risk_category STRING,

    

    -- Aggregated metrics

    total_claims BIGINT,

    total_claim_amount DECIMAL(20,2),

    avg_claim_amount DECIMAL(15,2),

    avg_fraud_score DOUBLE,

    high_value_claims BIGINT,

    unique_policies BIGINT,

    

    -- Processing metadata

    processing_date DATE,

    gold_version STRING

)

USING DELTA

CLUSTER BY (claim_year, claim_month, claim_type)

""")

print("Gold layer analytics table created successfully!")

# Create Gold layer ML-ready dataset

spark.sql("""

CREATE TABLE IF NOT EXISTS insurance.gold.fraud_ml_dataset (

    -- Core features

    policy_id STRING,

    claim_date TIMESTAMP,

    claim_amount DECIMAL(15,2),

    fraud_score INT,

    

    -- Categorical features (indexed)

    claim_type_index INT,

    incident_type_index INT,

    location_index INT,

    

    -- Temporal features

    claim_month INT,

    claim_day_of_week INT,

    claim_hour INT,

    

    -- Derived features

    amount_category STRING,

    is_high_value_claim BOOLEAN,

    is_fraud BOOLEAN,

    

    -- Processing metadata

    created_date DATE,

    dataset_version STRING

)

USING DELTA

CLUSTER BY (fraud_score, claim_date)

""")

print("Gold layer ML dataset table created successfully!")
print("Clustering optimized for fraud analysis and temporal patterns.")

Gold layer analytics table created successfully!


Gold layer ML dataset table created successfully!
Clustering optimized for fraud analysis and temporal patterns.


## Step 6: Transform Silver to Gold Layer Analytics



### Business Intelligence Aggregations


Create aggregated metrics for:


- **Temporal analysis**: Monthly/quarterly claim trends
- **Risk analysis**: Fraud patterns and risk distributions
- **Operational metrics**: Claim volumes and processing efficiency
- **Geographic insights**: Location-based claim patterns



### Feature Engineering for ML


Prepare features for fraud detection:


- **Categorical encoding**: Convert strings to numeric indices
- **Temporal features**: Extract time-based patterns
- **Derived features**: Business logic-based indicators
- **Target variable**: Fraud classification based on score thresholds

In [31]:
# Transform Silver data to Gold layer analytics

from pyspark.sql.functions import (
    count, sum, avg, countDistinct, current_date, lit,
    round as spark_round
)

# Read Silver layer data
df_silver = spark.read.table("insurance.silver.insurance_claims_clean")

print(f"Processing {df_silver.count()} records from Silver layer for Gold layer analytics")

# Create aggregated analytics dataset
df_analytics = df_silver.groupBy(
    "claim_year", "claim_month", "claim_type", 
    "incident_type", "location", "risk_category"
).agg(
    count("*").alias("total_claims"),
    sum("claim_amount").alias("total_claim_amount"),
    spark_round(avg("claim_amount"), 2).alias("avg_claim_amount"),
    spark_round(avg("fraud_score"), 2).alias("avg_fraud_score"),
    sum(when(col("is_high_value_claim"), 1).otherwise(0)).alias("high_value_claims"),
    countDistinct("policy_id").alias("unique_policies")
).withColumn("processing_date", current_date()) \
 .withColumn("gold_version", lit("v1.0"))

print(f"Created {df_analytics.count()} aggregated analytics records")

# Show sample analytics
print("\nSample Gold Layer Analytics:")
df_analytics.orderBy(col("total_claims").desc()).show(10)

Processing 9666 records from Silver layer for Gold layer analytics


Created 7397 aggregated analytics records

Sample Gold Layer Analytics:


+----------+-----------+----------+-----------------+---------------+--------------+------------+------------------+----------------+---------------+-----------------+---------------+---------------+------------+
|claim_year|claim_month|claim_type|    incident_type|       location| risk_category|total_claims|total_claim_amount|avg_claim_amount|avg_fraud_score|high_value_claims|unique_policies|processing_date|gold_version|
+----------+-----------+----------+-----------------+---------------+--------------+------------+------------------+----------------+---------------+-----------------+---------------+---------------+------------+
|      2024|          9|      Home|             Fire|   New York, NY|     High Risk|           5|         401940.56|        80388.11|           70.6|                2|              5|     2025-12-19|        v1.0|
|      2024|          1|    Health|Medical Emergency|     Denver, CO|     High Risk|           5|         203666.18|        40733.24|           68.4

In [32]:
# Insert analytics data into Gold layer

df_analytics.write.mode("overwrite").saveAsTable("insurance.gold.claim_analytics")

print(f"Successfully inserted {df_analytics.count()} analytics records into insurance.gold.claim_analytics")

# Verify Gold analytics data
analytics_summary = spark.sql("""
    SELECT 
        COUNT(*) as total_aggregations,
        ROUND(SUM(total_claim_amount), 2) as total_claim_value,
        ROUND(AVG(avg_fraud_score), 2) as avg_fraud_score
    FROM insurance.gold.claim_analytics
""").collect()[0]

print(f"\nGold layer analytics summary:")
print(f"- Total aggregations: {analytics_summary[0]}")
print(f"- Total claim value: ${analytics_summary[1]:,}")
print(f"- Average fraud score: {analytics_summary[2]}")

Successfully inserted 7397 analytics records into insurance.gold.claim_analytics



Gold layer analytics summary:
- Total aggregations: 7397
- Total claim value: $978,477,793.21
- Average fraud score: 49.28


## Step 7: Create ML-Ready Fraud Detection Dataset



### Feature Engineering Pipeline


Transform Silver data into ML-ready features:


1. **Categorical Encoding**: Convert categorical variables to numeric indices
2. **Feature Selection**: Choose relevant features for fraud detection
3. **Target Creation**: Define fraud based on score thresholds
4. **Data Splitting**: Prepare train/validation/test sets

In [27]:
# Create ML-ready fraud detection dataset

from pyspark.ml.feature import StringIndexer
from pyspark.sql.functions import current_date

# Read Silver layer data for ML
df_silver_ml = spark.read.table("insurance.silver.insurance_claims_clean")

# Create indexers for categorical variables
claim_type_indexer = StringIndexer(inputCol="claim_type", outputCol="claim_type_index")
incident_type_indexer = StringIndexer(inputCol="incident_type", outputCol="incident_type_index")
location_indexer = StringIndexer(inputCol="location", outputCol="location_index")

# Apply indexing
df_ml_indexed = claim_type_indexer.fit(df_silver_ml).transform(df_silver_ml)
df_ml_indexed = incident_type_indexer.fit(df_ml_indexed).transform(df_ml_indexed)
df_ml_indexed = location_indexer.fit(df_ml_indexed).transform(df_ml_indexed)

# Create ML dataset with selected features
df_ml_dataset = df_ml_indexed.select(
    "policy_id",
    "claim_date",
    "claim_amount",
    "fraud_score",
    "claim_type_index",
    "incident_type_index",
    "location_index",
    "claim_month",
    "claim_day_of_week",
    "claim_hour",
    "amount_category",
    "is_high_value_claim",
    (col("fraud_score") >= 60).alias("is_fraud"),
    current_date().alias("created_date"),
    lit("v1.0").alias("dataset_version")
)

print(f"Created ML-ready dataset with {df_ml_dataset.count()} records")
print("\nFraud distribution:")
df_ml_dataset.groupBy("is_fraud").count().show()

print("\nSample ML Dataset:")
df_ml_dataset.show(5)

Created ML-ready dataset with 9666 records

Fraud distribution:


+--------+-----+
|is_fraud|count|
+--------+-----+
|    true| 3850|
|   false| 5816|
+--------+-----+


Sample ML Dataset:


+-----------+-------------------+------------+-----------+----------------+-------------------+--------------+-----------+-----------------+----------+---------------+-------------------+--------+------------+---------------+
|  policy_id|         claim_date|claim_amount|fraud_score|claim_type_index|incident_type_index|location_index|claim_month|claim_day_of_week|claim_hour|amount_category|is_high_value_claim|is_fraud|created_date|dataset_version|
+-----------+-------------------+------------+-----------+----------------+-------------------+--------------+-----------+-----------------+----------+---------------+-------------------+--------+------------+---------------+
|POL00000001|2024-10-31 17:00:00|    30735.20|          8|             3.0|                0.0|           2.0|         10|                5|        17|   Medium Value|              false|   false|  2025-12-19|           v1.0|
|POL00000002|2024-07-28 22:00:00|   146227.66|         33|             4.0|                1.0| 

In [28]:
# Insert ML dataset into Gold layer

df_ml_dataset.write.mode("overwrite").saveAsTable("insurance.gold.fraud_ml_dataset")

print(f"Successfully inserted {df_ml_dataset.count()} ML-ready records into insurance.gold.fraud_ml_dataset")

# Verify Gold ML data
ml_summary = spark.sql("""
    SELECT 
        COUNT(*) as total_records,
        SUM(CASE WHEN is_fraud THEN 1 ELSE 0 END) as fraud_cases,
        ROUND(AVG(claim_amount), 2) as avg_claim_amount,
        ROUND(AVG(fraud_score), 2) as avg_fraud_score
    FROM insurance.gold.fraud_ml_dataset
""").collect()[0]

print(f"\nGold layer ML dataset summary:")
print(f"- Total records: {ml_summary[0]}")
print(f"- Fraud cases: {ml_summary[1]} ({ml_summary[1]/ml_summary[0]*100:.1f}%)")
print(f"- Average claim amount: ${ml_summary[2]:,}")
print(f"- Average fraud score: {ml_summary[3]}")

Successfully inserted 9666 ML-ready records into insurance.gold.fraud_ml_dataset



Gold layer ML dataset summary:
- Total records: 9666
- Fraud cases: 3850 (39.8%)
- Average claim amount: $101,228.82
- Average fraud score: 49.49


## Step 8: Demonstrate Medallion Architecture Benefits



### Query Performance Across Layers


Show how liquid clustering optimizes queries at each layer:


- **Bronze**: Raw data queries with ingestion metadata
- **Silver**: Clean data queries with derived fields
- **Gold**: Aggregated analytics and ML-ready queries



### Business Value Demonstration


Illustrate insurance analytics use cases:


- **Operational monitoring**: Claim processing metrics
- **Risk analysis**: Fraud pattern detection
- **Business intelligence**: Trend analysis and KPIs

In [33]:
# Demonstrate Medallion Architecture query performance

print("=== Medallion Architecture Query Performance Demo ===\n")

# Bronze Layer: Raw data analysis
print("1. BRONZE LAYER - Raw Data Ingestion Analysis")
bronze_stats = spark.sql("""
    SELECT 
        source_system,
        COUNT(*) as records_ingested,
        ROUND(AVG(claim_amount), 2) as avg_amount
    FROM insurance.bronze.insurance_claims_raw
    GROUP BY source_system
    ORDER BY records_ingested DESC
""")
bronze_stats.show()

# Silver Layer: Clean data analysis
print("\n2. SILVER LAYER - Clean Data Quality Analysis")
silver_stats = spark.sql("""
    SELECT 
        risk_category,
        COUNT(*) as claims,
        ROUND(AVG(claim_amount), 2) as avg_amount,
        ROUND(AVG(data_quality_score), 3) as quality_score
    FROM insurance.silver.insurance_claims_clean
    GROUP BY risk_category
    ORDER BY claims DESC
""")
silver_stats.show()

# Gold Layer: Business analytics
print("\n3. GOLD LAYER - Business Intelligence Analytics")
gold_stats = spark.sql("""
    SELECT 
        claim_year,
        claim_month,
        SUM(total_claims) as monthly_claims,
        ROUND(SUM(total_claim_amount), 2) as monthly_amount,
        ROUND(AVG(avg_fraud_score), 2) as avg_risk_score
    FROM insurance.gold.claim_analytics
    GROUP BY claim_year, claim_month
    ORDER BY claim_year, claim_month
""")
gold_stats.show()

# Gold Layer: ML insights
print("\n4. GOLD LAYER - ML-Ready Fraud Analysis")
fraud_insights = spark.sql("""
    SELECT 
        amount_category,
        is_fraud,
        COUNT(*) as claims,
        ROUND(AVG(claim_amount), 2) as avg_amount
    FROM insurance.gold.fraud_ml_dataset
    GROUP BY amount_category, is_fraud
    ORDER BY amount_category, is_fraud
""")
fraud_insights.show()

=== Medallion Architecture Query Performance Demo ===

1. BRONZE LAYER - Raw Data Ingestion Analysis


+-------------+----------------+----------+
|source_system|records_ingested|avg_amount|
+-------------+----------------+----------+
|Legacy_Claims|            5180|  79355.76|
|  Partner_API|            2274| 102191.12|
|   Mobile_App|            1312| 104661.70|
|   Direct_Web|             900|  99272.45|
+-------------+----------------+----------+


2. SILVER LAYER - Clean Data Quality Analysis


+--------------+------+----------+-------------+
| risk_category|claims|avg_amount|quality_score|
+--------------+------+----------+-------------+
|   Medium Risk|  1956| 102039.71|          1.0|
|     High Risk|  1946| 103120.08|          1.0|
|      Low Risk|  1940|  96515.19|          1.0|
| Very Low Risk|  1920| 104572.57|          1.0|
|Very High Risk|  1904|  99893.72|          1.0|
+--------------+------+----------+-------------+


3. GOLD LAYER - Business Intelligence Analytics


+----------+-----------+--------------+--------------+--------------+
|claim_year|claim_month|monthly_claims|monthly_amount|avg_risk_score|
+----------+-----------+--------------+--------------+--------------+
|      2024|          1|           819|   80797908.54|         48.93|
|      2024|          2|           773|   76222034.76|         50.82|
|      2024|          3|           800|   81043424.47|         49.08|
|      2024|          4|           816|   81202422.92|         48.85|
|      2024|          5|           825|   82443145.34|         48.94|
|      2024|          6|           770|   77217884.04|         50.08|
|      2024|          7|           795|   80651164.65|         48.33|
|      2024|          8|           828|   84490675.35|         48.71|
|      2024|          9|           780|   79059662.50|         49.04|
|      2024|         10|           816|   85649684.06|         49.98|
|      2024|         11|           830|   85716244.97|         49.97|
|      2024|        

+---------------+--------+------+----------+
|amount_category|is_fraud|claims|avg_amount|
+---------------+--------+------+----------+
|     High Value|   false|  1945| 215219.15|
|     High Value|    true|  1291| 217092.81|
|      Low Value|   false|  1191|  13202.54|
|      Low Value|    true|   816|  13251.86|
|   Medium Value|   false|  2680|  57195.15|
|   Medium Value|    true|  1743|  57251.28|
+---------------+--------+------+----------+



## Step 9: Train Fraud Detection Model on Gold Layer Data



### Machine Learning on Curated Data


Use the Gold layer ML-ready dataset to train a fraud detection model:


- **Feature-rich**: Engineered features from Silver layer
- **Optimized**: Liquid clustering for fast model training
- **Business-focused**: Fraud prediction with clear business impact



### Model Evaluation and Business Impact


Evaluate model performance and calculate:


- **Accuracy metrics**: Precision, recall, AUC
- **Business value**: Potential fraud detection savings
- **Operational impact**: Investigation prioritization

In [34]:
# Train fraud detection model using Gold layer ML dataset

from pyspark.ml.feature import VectorAssembler, StandardScaler
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml import Pipeline
import pyspark.sql.functions as F

# Load Gold layer ML dataset
ml_data = spark.read.table("insurance.gold.fraud_ml_dataset")

print(f"Loaded {ml_data.count()} records from Gold layer ML dataset")

# Prepare features for ML
feature_cols = [
    "claim_amount", "claim_month", "claim_day_of_week", "claim_hour",
    "claim_type_index", "incident_type_index", "location_index"
]

# Create feature vector
assembler = VectorAssembler(inputCols=feature_cols, outputCol="features")
scaler = StandardScaler(inputCol="features", outputCol="scaled_features")

# Train Random Forest model
rf = RandomForestClassifier(
    labelCol="is_fraud",
    featuresCol="scaled_features",
    numTrees=100,
    maxDepth=10,
    seed=42
)

# Create pipeline
pipeline = Pipeline(stages=[assembler, scaler, rf])

# Split data
train_data, test_data = ml_data.randomSplit([0.8, 0.2], seed=42)

print(f"Training set: {train_data.count()} records")
print(f"Test set: {test_data.count()} records")

# Show class distribution
print("\nFraud distribution in training set:")
train_data.groupBy("is_fraud").count().show()

Loaded 9666 records from Gold layer ML dataset


Training set: 7807 records


Test set: 1859 records

Fraud distribution in training set:


+--------+-----+
|is_fraud|count|
+--------+-----+
|    true| 3087|
|   false| 4720|
+--------+-----+



In [36]:
# Train and evaluate the fraud detection model
train_data = train_data.withColumn("is_fraud", col("is_fraud").cast(IntegerType()))
test_data = test_data.withColumn("is_fraud", col("is_fraud").cast(IntegerType()))

print("Training fraud detection model on Gold layer data...")
model = pipeline.fit(train_data)

# Make predictions
predictions = model.transform(test_data)

# Evaluate model
evaluator = BinaryClassificationEvaluator(labelCol="is_fraud", metricName="areaUnderROC")
auc = evaluator.evaluate(predictions)

print(f"\nModel Performance on Gold Layer Data:")
print(f"AUC: {auc:.4f}")

# Calculate detailed metrics
tp = predictions.filter("is_fraud = 1 AND prediction = 1").count()
tn = predictions.filter("is_fraud = 0 AND prediction = 0").count()
fp = predictions.filter("is_fraud = 0 AND prediction = 1").count()
fn = predictions.filter("is_fraud = 1 AND prediction = 0").count()

precision = tp / (tp + fp) if (tp + fp) > 0 else 0
recall = tp / (tp + fn) if (tp + fn) > 0 else 0
accuracy = (tp + tn) / (tp + tn + fp + fn)

print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"Accuracy: {accuracy:.4f}")

# Business impact analysis
fraud_claims = predictions.filter("prediction = 1")
potential_fraud_value = fraud_claims.agg(F.sum("claim_amount")).collect()[0][0]
total_test_value = test_data.agg(F.sum("claim_amount")).collect()[0][0]

print(f"\nBusiness Impact Analysis:")
print(f"Total test set claim value: ${total_test_value:,.2f}")
print(f"Predicted fraudulent claims value: ${potential_fraud_value:,.2f}")
print(f"Fraud detection coverage: {potential_fraud_value/total_test_value*100:.1f}%")
print(f"Potential annual savings (est.): ${potential_fraud_value*12:,.2f}")

# Show sample predictions
print("\nSample Fraud Predictions:")
predictions.select("policy_id", "claim_amount", "fraud_score", "is_fraud", "prediction", "probability") \
          .orderBy(F.desc("probability")) \
          .show(10)

Training fraud detection model on Gold layer data...



Model Performance on Gold Layer Data:
AUC: 0.4979


Precision: 0.3736
Recall: 0.0446
Accuracy: 0.5772



Business Impact Analysis:
Total test set claim value: $184,631,848.85
Predicted fraudulent claims value: $12,207,875.25
Fraud detection coverage: 6.6%
Potential annual savings (est.): $146,494,503.00

Sample Fraud Predictions:


+-----------+------------+-----------+--------+----------+--------------------+
|  policy_id|claim_amount|fraud_score|is_fraud|prediction|         probability|
+-----------+------------+-----------+--------+----------+--------------------+
|POL00009569|    12449.93|         65|       1|       0.0|[0.79989178380132...|
|POL00008076|     1366.26|         34|       0|       0.0|[0.77271661548148...|
|POL00008940|    30951.86|         84|       1|       0.0|[0.75911306876166...|
|POL00006015|   109689.57|         49|       0|       0.0|[0.75803257555286...|
|POL00009235|    43158.71|         91|       1|       0.0|[0.73937684219733...|
|POL00001855|     8171.97|         99|       1|       0.0|[0.73877500051399...|
|POL00001810|   143294.36|         97|       1|       0.0|[0.73588989728601...|
|POL00009662|   220301.52|         29|       0|       0.0|[0.73233852437669...|
|POL00002310|     8861.38|         49|       0|       0.0|[0.73077833978262...|
|POL00009034|   162076.57|          4|  

# Key Takeaways: Medallion Architecture with Delta Liquid Clustering



## What We Demonstrated



### 1. Complete Medallion Architecture Implementation


- **Bronze Layer**: Raw data ingestion with liquid clustering on `(policy_id, ingestion_date)`
- **Silver Layer**: Data quality, cleansing, and enrichment with clustering on `(policy_id, claim_date)`
- **Gold Layer**: Business analytics and ML-ready datasets with specialized clustering


### 2. Delta Liquid Clustering Optimization


- **Automatic optimization**: No manual partitioning or Z-Ordering required
- **Query-specific clustering**: Each layer optimized for its access patterns
- **Performance benefits**: Fast queries on clustered columns
- **Maintenance-free**: Automatic data layout optimization


### 3. Progressive Data Refinement


- **Bronze**: Preserves data fidelity with ingestion metadata
- **Silver**: Clean, standardized data with derived fields
- **Gold**: Curated analytics and ML-ready features


### 4. Insurance Business Value


- **Fraud Detection**: ML model trained on curated Gold layer data
- **Operational Analytics**: Real-time claim processing insights
- **Risk Management**: Automated risk scoring and categorization
- **Business Intelligence**: Trend analysis and KPI monitoring


## AIDP Advantages


- **Unified Platform**: Seamless data flow from ingestion to analytics
- **Performance**: Liquid clustering optimizes each layer for its use case
- **Governance**: Catalog-based data organization and access control
- **Scalability**: Handles insurance-scale data volumes efficiently


## Best Practices Demonstrated


### Data Architecture
- Choose clustering columns based on primary query patterns
- Design layer-specific schemas for optimal performance
- Implement proper data quality checks and monitoring

### Performance Optimization
- Use liquid clustering for automatic data layout optimization
- Align clustering strategy with access patterns
- Balance between too few and too many clustering columns

### Business Value
- Focus on end-to-end analytics workflows
- Enable self-service analytics with curated datasets
- Integrate ML capabilities for predictive insights


## Next Steps


- **Scale Up**: Process larger insurance datasets
- **Add Streaming**: Implement real-time claim processing
- **Advanced ML**: Try AutoML and deep learning models
- **Data Products**: Create APIs for claim analytics
- **Monitoring**: Add data quality and performance monitoring


This implementation demonstrates how Oracle AI Data Platform enables sophisticated insurance analytics while maintaining enterprise-grade performance, governance, and scalability.